In [1]:
from tdmclient import ClientAsync, aw
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

Node 83c7a2dd-bef5-47dc-bc6b-3a4b432d84e8

In [2]:
import vision as vis
import kalman
import numpy as np
import time
import cv2 as cv
import shortest_path as short
import positions as pos
import math
import motion_control as motion



COMPUTE = 0
TRAVEL = 1
LOCAL_NAV = 2
FINISH = 3

In [3]:
# Main -> state machine
state = COMPUTE
obstacle = False
path_calculated = False

#valeurs pour initialiser
robot_pos_kalman = np.array([[0],[0],[0]]) #3-by-1 np.array
Sigma_kalman = np.diag([0.01,0.01,0.01])
HISTORY_SIZE = 15
history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE

cap = cv.VideoCapture(2)  
cap.set(cv.CAP_PROP_AUTO_EXPOSURE, 1)
while True:
    if state == COMPUTE:
        captured, img = cap.read()
        if captured:
            cropped, cropped_img = vis.crop_map(img)
            # print(cropped_img.shape)        # uncomment to debug
            # cv.imshow("Original", img)        # uncomment to debug
            cv.imshow("Cropped", cropped_img)
            if cropped:
                im3 = np.zeros(cropped_img.shape)
                found, obstacles = vis.extract_obstacles(cropped_img)
                if found:
                    ex_obstacles = vis.expand_obstacles(obstacles)
                    cv.drawContours(im3, ex_obstacles, -1, (0, 255, 0), 3)
                    cv.imshow("expanded", im3)
                found_rob, robot_pos = pos.get_robot_position(cropped_img)
                if found_rob:
                    vis.annotate_robot(robot_pos, im3)
                found_goal, goal_pos, radius_pxl = pos.get_goal_position(cropped_img)
                if found_goal:
                    vis.annotate_goal(goal_pos, im3)
                found_arch, arch_pos = pos.get_arch_positions(cropped_img)
                if found_arch and found_rob:
                    if math.dist(arch_pos[0],robot_pos[0])>math.dist(arch_pos[1],robot_pos[0]):
                        arch_pos = np.flip(arch_pos,0)
                    vis.annotate_arch(arch_pos, im3)
                if (found and found_rob and found_goal and found_arch):
                    scale_factor = pos.convert_meter2pxl(radius_pxl)
                    if path_calculated == False:
                        vertices = vis.convert_vertice(ex_obstacles)
                        pathname = short.find_shortest_path(vertices, tuple(robot_pos[0]), tuple(arch_pos[0]))
                        path_arch =\
                        short.names_to_subpaths(pathname,vertices,tuple(robot_pos[0]), tuple(arch_pos[0]))[0]
                        vis.annotate_path(path_arch,im3)

                        pathname2 = short.find_shortest_path(vertices, tuple(arch_pos[1]), tuple(goal_pos))
                        path_goal =\
                        short.names_to_subpaths(pathname2,vertices,tuple(arch_pos[1]), tuple(goal_pos))[0]
                        vis.annotate_path(path_goal,im3)
                        cv.imshow("environment", im3)
                        
                        #initialize kalman filter
                        robot_pos_kalman = np.array([[robot_pos[0][0]],[robot_pos[0][1]],[robot_pos[1]]]) #3-by-1 np.array
                        HISTORY_SIZE = 15
                        history_kalman = [np.reshape(robot_pos_kalman,(1,3))]*HISTORY_SIZE
                        
                        state = TRAVEL
        else:
            print("There was a problem in the capture")
            break
    if state == TRAVEL:
        cv.imshow("environment", im3)
        cv.waitKey(0)
        for subgoal in path_arch[1:]:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos,im4)
                vis.annotate_path(path_arch,im4)
                
                
                #read values from camera & sensors
                success_camera, robot_pos_camera = pos.get_robot_position(cropped_travel)
                #print("robot angle", robot_pos_camera[1])
                if sucess_camera:
                    camera_input = [(robot_pos_camera[0][0],robot_pos_camera[0][1]), robot_pos_camera[1], success_camera]
                else:
                    camera_input = [(0,0),0,False]
                await node.wait_for_variables({"motor.left.speed"})
                await node.wait_for_variables({"motor.right.speed"})
                motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        
                Ts = 0.01 #how to check this?
                
                #get position from kalman filter
                robot_pos_kalman, Sigma_kalman, history_kalman =\
                kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,history_kalman, camera_input, Ts, scale_factor)
                robot_pos = [(robot_pos_kalman[0][0], robot_pos_kalman[1][0]), robot_pos_kalman[2][0]]
                
                #et apres, on passe robot_pos au controleur
                
                
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(1)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        for subgoal in path_goal:
            while not motion.check_robot_arrived(robot_pos,subgoal):
                captured, img_travel = cap.read()
                cropped, cropped_travel = vis.crop_map(img_travel)
                cv.imshow("cropped",cropped_travel)
                im4 = np.zeros(cropped_travel.shape)
                vis.annotate_robot(robot_pos,im4)
                vis.annotate_path(path_goal,im4)
                #print("robot angle", robot_pos[1])
                
                #read values from camera & sensors
                success_camera, robot_pos_camera = pos.get_robot_position(cropped_travel)
                if sucess_camera:
                    camera_input = [(robot_pos_camera[0][0],robot_pos_camera[0][1]), robot_pos_camera[1], success_camera]
                else:
                    camera_input = [(0,0),0,False]
                await node.wait_for_variables({"motor.left.speed"})
                await node.wait_for_variables({"motor.right.speed"})
                motorspeed = [node.v.motor.right.speed, node.v.motor.left.speed]        
                Ts = 0.01 #how to check this?
                
                #get position from kalman filter
                robot_pos_kalman, Sigma_kalman, history_kalman =\
                kalman.kalmanfilter(robot_pos_kalman,Sigma_kalman,motorspeed,history_kalman, camera_input, Ts, scale_factor)
                robot_pos = [(robot_pos_kalman[0][0], robot_pos_kalman[1][0]), robot_pos_kalman[2][0]]
                
                
                err = motion.get_error(robot_pos,subgoal)
                cv.putText(im4, str(err), (40, 40), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.imshow("mission status", im4)
                cv.waitKey(1)
                motor_left, motor_right = motion.speed_control(err)
                motor_left = round(motor_left)
                motor_right = round(motor_right)
                v = {
                    "motor.left.target": [motor_left],
                    "motor.right.target": [motor_right],
                }
                aw(node.set_variables(v))
        state = FINISH
    if state == FINISH:
        v = {
                    "motor.left.target": [0],
                    "motor.right.target": [0],
                }
        aw(node.set_variables(v))
            #print("motor_left",motor_left)
            #print("motor_right", motor_right)
            #motor_left_target = int(motor_left)
            #motor_right_target = int(motor_right)
            #print("motor_left_t",motor_left_target)
            #print("motor_right_t", motor_right_target)          
    cv.imshow("environment",im3)
    #cv.waitKey(0)
cap.release()
print("finished!")
# cv.destroyAllWindows()
    

"""
# robot run    
elif short.find_shortest_path() == True:
    state = 1

obstacle = check_obstacle(prox_horizontal)
    
        # global navigation
        if  obstacle == False:
            motion_control

        # local navigation
        elif obstacle == True:
"""
            
            
        
    

C:\Users\sebas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\lib\function_base.py:804: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, order=order, subok=subok, copy=True)


Start (546, 332)
Goal (272, 293)
Vertices [[(242, 318), (386, 321), (386, 449), (242, 449)], [(225, 144), (284, 172), (180, 384), (132, 357)], [(447, 119), (381, 305), (311, 163)], [(129, -20), (220, 77), (119, 172), (20, 76)]]
list neighbours [[(2, 160.37767924496225), (3, 198.2145302443794), (9, 234.8829495727606), (10, 167.1944975171133), (16, 276.76163028859327)], [(2, 144.03124660989366), (4, 131.0), (6, 151.92103211866353), (7, 90.55385138137417), (10, 139.60659010233005), (11, 169.66437457521835), (16, 39.05124837953327)], [(0, 160.37767924496225), (1, 144.03124660989366), (3, 128.0), (6, 180.56854654119581), (9, 211.00947846009194), (10, 16.76305461424021), (11, 174.89711261195822), (13, 295.1135374732918), (16, 117.38824472663352)], [(0, 198.2145302443794), (2, 128.0), (4, 144.0), (9, 335.59052430007614)], [(1, 131.0), (3, 144.0), (7, 89.8276126811795), (8, 143.4015341619468)], [(6, 65.30696746902278), (8, 232.41772737895877), (9, 223.40322289528413), (11, 88.07383266328314), 

TypeError: 'NoneType' object is not subscriptable

In [ ]:
OBSTACLE_THRESHOLD 10

def check_obstacle(prox_horizontal):
    prox_values = [prox_horizontal[0], prox_horizontal[4]]
    if prox_values[0] > OBSTACLE_THRESHOLD or prox_values[1] > OBSTACLE_THRESHOLD:
        obstacle = True
        
    else:
        obstacle = False
        
        
    